### Imports and training Configurations

In [ ]:
import torch
import syft as sy
import argparse
import numpy as np
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torchvision import transforms
from torchvision import datasets, transforms

import time
print("torch version:", torch.__version__)
print("syft version:", sy.__version__)

#All warnings resulting from running these imports are normal due to tensorflow moving from 1.x to 2.0 as well as numpy deprecating a few commands

In [ ]:
def getSamples(filename):
    data = pd.read_csv(filename, sep='\t')
    return data.values[:,1:].transpose()

dim = 12634
data1 = getSamples("GSE2034-Normal-train.txt")
data2 = getSamples("GSE2034-Tumor-train.txt")

data1Label = np.zeros(len(data1)).reshape((-1, 1))
data2Label = np.ones(len(data2)).reshape((-1, 1))
x = np.concatenate((data1, data2))
y = np.concatenate((data1Label, data2Label))

# shuffle the data
idx = np.random.permutation(len(x))
x,y = x[idx], y[idx]

z = np.concatenate((x, y), axis = 1)

# We follow an 80/20 partitioning for the training and testing sets
n_train_items = 181
n_test_items = 46

# partition the data into training data and test data
x_train = x[:n_train_items]
y_train = y[:n_train_items]

x_train = x_train.reshape((-1,1,dim))
y_train = y_train.reshape((-1,1))

x_test = x[n_train_items:]
y_test = y[n_train_items:]    

x_test = x_test.reshape((-1,1,dim))
y_test = y_test.reshape((-1,1,1))

In [ ]:
# get the data into (batch, channel = 1, length=dim)
data_torch = torch.from_numpy(x).view([-1, 1, dim]).float()
label_torch = torch.from_numpy(y).view([-1,1,1]).float()

### Deep Learning Model applied to GSE2034

In [ ]:
dim = 12634

class Res1d(nn.Module):
    # the conv layers
    def __init__(self, inSize, outSize, kernel=(3,), strides=1,):
        super(Res1d, self).__init__()
        # hard-coded to do the padding correctly
        if inSize in (16,64,128,512) and strides is 2:
            pding = 0
        else:
            pding = 1
        self.l1 = nn.Conv1d(inSize, outSize, kernel, stride=strides, padding=pding, bias=False)
        self.l2 = nn.InstanceNorm1d(outSize)
        
        if strides > 1 or inSize != outSize:
            if strides > 1:
                self.r1 = nn.Identity()
                self.r2 = nn.AvgPool1d(strides)
            else:
                self.r1 = None
                self.r2 = None
            self.r3 = nn.Conv1d(inSize, outSize, 1, bias=False)
            self.r4 = nn.InstanceNorm1d(outSize)
    
        self.relu = nn.ReLU()
        
    def forward(self, x):
        l = self.l1(x)
        l = self.l2(l)
        
        if self.r1 is not None:
            r = self.r1(x)
            r = self.r2(r)
            r = self.r3(r)
            r = self.r4(r)
        else:
            r = self.r3(x)
            r = self.r4(r)
            
        x = l + r
        return self.relu(x)
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(dim, 64)
        self.l2 = nn.ReLU()
        
        self.r1 = Res1d(1, 4, 3)
        
        self.r2 = Res1d(4, 8, 3)
        self.r3 = Res1d(8, 8, 3, strides=2)
        
        self.r4 = Res1d(8, 16, 3)
        self.r5 = Res1d(16, 16, 3, strides=2)
        
        self.r6 = Res1d(16, 32, 3)
        self.r7 = Res1d(32, 32, 3, strides=2)
        
        self.r8 = Res1d(32, 64, 3)
        self.r9 = Res1d(64, 64, 3, strides=2)
        
        self.r10 = Res1d(64, 128, 3)
        self.r11 = Res1d(128, 128, 3, strides=2)
        
        self.r12 = Res1d(128, 256, 3)
        self.r13 = Res1d(256, 256, 3, strides=2)
        
        self.r14 = Res1d(256, 512, 3)
        self.r15 = Res1d(512, 512, 3, strides=2)
        
        self.r16 = Res1d(512, 1024, 3)
        self.r17 = Res1d(1024, 1024, 3, strides=2)
        
        # size is by experiment and hardcode
        self.lastLinear = nn.Linear(50240,32)
        self.lastRelu = nn.ReLU()
        self.lastAgg = nn.Linear(32,1)
        self.lastSigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # shape is (batch, channel, time)
        l = x
        l = self.l2(self.l1(l))
        
        # conv layers should operate on time
        r = x
        r = self.r4(self.r3(self.r2(self.r1(r))))
        r = self.r8(self.r7(self.r6(self.r5(r))))
        r = self.r12(self.r11(self.r10(self.r9(r))))
        r = self.r16(self.r15(self.r14(self.r13(r))))
        r = self.r17(r)
        
        # flatten l
        r = r.view(x.shape[0],1, -1)
        y = torch.cat((l,r),-1)
        y = self.lastLinear(y)
        y = self.lastRelu(y)
        y = self.lastAgg(y)
        y = self.lastSigmoid(y)
        return y

In [ ]:
device = torch.device("cuda")
net = Net().to(device)

In [ ]:
import torch.optim as optim
criterion = nn.BCELoss() # Binary Cross Entropy 
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Training on plain data

In [ ]:
test_inputs = torch.from_numpy(x_test).view([-1, 1, dim]).float().cuda()
test_labels = torch.from_numpy(y_test).view([-1, 1]).float().cuda()
for batch in range(100):  # loop over the dataset multiple times
    # get the inputs; data is a list of [inputs, labels]
    indices = np.random.choice(len(x_train), size=(30))
    inputs = x_train[indices]
    labels = y_train[indices]
    
    inputs = torch.from_numpy(inputs).float().cuda()
    labels = torch.from_numpy(labels).float().cuda()
    
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs).view([-1,1]).cuda()
    loss = criterion(outputs, labels).cuda()
    loss.backward()
    optimizer.step()
    
    print(batch, "% Trained", "loss = ", loss.item())

print('Finished Training')

In [ ]:
net

In [ ]:
net(data_torch)

### Setup of the secure environment

In [ ]:
hook = sy.TorchHook(torch)

def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]

def connect_to_crypto_provider():
    return sy.VirtualWorker(hook, id="crypto_provider")

workers = connect_to_workers(n_workers=2)
crypto_provider = connect_to_crypto_provider()

def get_private_data_loaders(precision_fractional, workers, crypto_provider):
    
    def secret_share(tensor): #Transforms to fixed precision and secret share a tensor
        return (
            tensor
            .fix_precision(precision_fractional=precision_fractional)
            .share(*workers, crypto_provider=crypto_provider, requires_grad=True)
        )
    
    private_train_loader = [
        (secret_share(torch.Tensor(x_train[i*5:i*5+5])), secret_share(torch.Tensor(y_train[i*5:i*5+5])))
        for i in range (n_train_items)
        if i < n_train_items / 5
    ]
    
    private_test_loader = [
        (secret_share(torch.Tensor(x_test[i*5:i*5+5])), secret_share(torch.Tensor(y_test[i*5:i*5+5])))
        for i in range (n_train_items)
        if i < n_train_items / 5
    ]
    return private_train_loader, private_test_loader
    
    
private_train_loader, private_test_loader = get_private_data_loaders(
    precision_fractional=3,
    workers=workers,
    crypto_provider=crypto_provider
)

### Model for encrypted data

In [ ]:
class Res1d(nn.Module):
    # the conv layers
    def __init__(self, inSize, outSize, kernel=(3,), strides=1,):
        super(Res1d, self).__init__()
        self.inSize = inSize
        self.outSize = outSize
        # hard-coded to do the padding correctly
        if inSize in (16,64,128,512) and strides is 2:
            pding = 0
        else:
            pding = 1
        self.l1 = nn.Conv1d(inSize, outSize, kernel, stride=strides, padding=pding, bias=False)
        self.l2 = nn.Identity()
        
        if strides > 1 or inSize != outSize:
            if strides > 1:
                self.r1 = nn.Identity()
                self.r2 = nn.AvgPool1d(strides)
            else:
                self.r1 = None
                self.r2 = None
            self.r3 = nn.Conv1d(inSize, outSize, 1, bias=False)
            self.r4 = nn.Identity()
            
    def forward(self, x):
        l = x
        l = self.l1(l)
        l = self.l2(l)
        
        if self.r1 is not None:
            r = self.r1(x)
            r = self.r2(r)
            r = self.r3(r)
            r = self.r4(r)
        else:
            r = self.r3(x)
            r = self.r4(r)
            
        x = l + r
        print("forwarding: ", self.inSize, self.outSize)
        return F.relu(x)
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(dim, 64)
        self.l2 = F.relu
        
        self.r1 = Res1d(1, 4, 3)
        
        self.r2 = Res1d(4, 8, 3)
        self.r3 = Res1d(8, 8, 3, strides=2)
        
        self.r4 = Res1d(8, 16, 3)
        self.r5 = Res1d(16, 16, 3, strides=2)
        
        self.r6 = Res1d(16, 32, 3)
        self.r7 = Res1d(32, 32, 3, strides=2)
        
        self.r8 = Res1d(32, 64, 3)
        self.r9 = Res1d(64, 64, 3, strides=2)
        
        self.r10 = Res1d(64, 128, 3)
        self.r11 = Res1d(128, 128, 3, strides=2)
        
        self.r12 = Res1d(128, 256, 3)
        self.r13 = Res1d(256, 256, 3, strides=2)
        
        self.r14 = Res1d(256, 512, 3)
        self.r15 = Res1d(512, 512, 3, strides=2)
        
        self.r16 = Res1d(512, 1024, 3)
        self.r17 = Res1d(1024, 1024, 3, strides=2)
        
        # size is by experiment and hardcode
        self.lastLinear = nn.Linear(50240,32)
        self.lastRelu = F.relu
        self.lastAgg = nn.Linear(32,1)
        self.lastSigmoid = nn.Sigmoid()
    
    def forward(self, x):           
        # shape is (batch, channel, time)
        l = x
        l = x.view(x.shape[0],-1)
        l = self.l1(l)
        l = self.l2(l)

        # conv layers should operate on time
        r = x
        r = self.r1(r)
        r = self.r4(self.r3(self.r2(r)))
        r = self.r8(self.r7(self.r6(self.r5(r))))
        r = self.r12(self.r11(self.r10(self.r9(r))))
        r = self.r16(self.r15(self.r14(self.r13(r))))
        r = self.r17(r)
        
        # flatten l
        r = r.view(x.shape[0],-1)
        l = l.view(x.shape[0],-1)
        y = torch.cat((l,r),dim=1)
        y = self.lastLinear(y)
        y = self.lastRelu(y)
        y = self.lastAgg(y)
        y = self.lastSigmoid(y)
        return y

### Private Training of the model

In [ ]:
def train(model, private_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(private_train_loader): # <-- now it is a private dataset
        print('training...')
        start_time = time.time()
        optimizer.zero_grad()
        
        output = model(data)
        
        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size
        
        loss.backward()
        
        optimizer.step()

        
#Although the aim of this project was to provide private training, we prepared code for testing as well
def test(model, private_test_loader):
    print('testing...')
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in private_test_loader:
            start_time = time.time()
            
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum()

    correct = correct.get().float_precision()

In [ ]:
model = Net()
model = model.fix_precision().share(*workers, crypto_provider=crypto_provider, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=0.001)
optimizer = optimizer.fix_precision() 
for epoch in range(1, 2):
    train(model, private_train_loader, optimizer, epoch)
    test(model, private_test_loader)